# Convert a z3d file to a txt file

## 1: Import modules

In [ ]:
import sys, os
import numpy as np
import pandas as pd

Get the path to the DoZen folder, which in this case is in the parent directory. Add DoZen to the path, so that we can import it

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
print(module_path)

In [ ]:
if module_path not in sys.path:
    sys.path.append(module_path)

At some point, I'll make DoZen importable with one line. In the meantime, import .py files as separate modules.

In [ ]:
# import DoZen functions
import z3dio
import timeio
import z3d_directory
import process

## 2. Specify filepaths

In [ ]:
#Enter the path to the z3d file.
z3d_path='/path/to/z3d_file.z3d'
# path to directory containing all calibration files
cal_dir = '/calibration_dir/'
# path to antenna.cal
antcal_file = '/path/to/antenna.cal'
# path to output text file
z3d_txt = '/path/to/calibrated.txt'

## 3. Read z3d file

In [ ]:
#Read the file contents.
z3d = z3dio.read_z3d(z3d_path)
print(z3d.keys())

The z3d object has a lot of info in it: metadata, data, times (only one per second), plus some other stuff

## 4. Create array of timestamps

Since gps_times are only once per second, we have to make our own array if we want every time in it

In [ ]:
if z3d['num_records']==0:
    valid = False
else:
    [dt_start,dt_end] = timeio.get_start_and_end_times_mountain(z3d,include_final_second=True,astype='timestamp')
    dt_times = pd.date_range(start=dt_start, 
                             end=dt_end, 
                             periods=len(z3d['data'])
                            )
    print(dt_times)

dt_times is a pandas DatetimeIndex. It is timezone-aware. It's just an evenly spaced array of timestamps. Writing it out to text will take a lot of space, though. Only write it to text if you need it.

## 5. Apply calibrations.

Read in all calibration files. There is a separate calibration file per Zen, and one master file for all mag antennas.

In [ ]:
# Read calibration files
# set ask_dir to True if you want to browse to the directory
cals = z3d_directory.read_zen_cals(cal_dir,ask_dir=False)
antcal = z3dio.read_antcal(antcal_file)

To apply calibrations, we'll need to move into the Fourier domain first.

In [ ]:
# must first interpolate over nan values in order to Fourier transform
data = z3d['data'].copy()
# ignore the first two seconds and the last second, which usually have nan values
data = data[8192:-4096]
times = dt_times[8192:-4096]
# check for nans
nans = np.isnan(data)
print(np.sum(nans),' NaN values')
if np.sum(nans)>0:
    # interpolate over nans
    ttt = lambda z: z.nonzero()[0]
    data[nans] = np.interp(ttt(nans),ttt(~nans),z3d['data'][~nans])

In [ ]:
# compute Fourier transform for real input
sampling_rate = z3d['metadata']['A/D Rate']
ft = np.fft.rfft(data)
freqs = np.fft.rfftfreq(len(data),1./sampling_rate)

Now, find the right calibrations, and apply them.

In [ ]:
# get metadata from z3d object
box_number = z3d['metadata']['Box number']
card_number = z3d['metadata']['ChannelSerial']
component = z3d['metadata']['CH.CMP']
# find the calibration for this Zen box number
zen_cal = cals.loc[box_number]
# apply the board calibration
ft_boardcal = process.apply_board_cal(ft,freqs,zen_cal,card_number,sampling_rate)

# apply mag antenna calibration, if applicable
if component[0] == 'H':
    antenna_number = z3d['metadata']['CH.ANTSN']
    ft_cal = process.apply_antenna_cal(ft_boardcal,freqs,antcal,antenna_number)
    # replace nan at zero-frequency with board calibrated value
    #ft_cal[0] = ft_boardcal[0]
    ft_cal[0] = 0
else:
    ft_cal = ft_boardcal
    
# inverse Fourier transform
data_cal = np.fft.irfft(ft_cal,len(data))

# Write calibrated time series to a text file

In [ ]:
np.savetxt(z3d_txt,data_cal)

Resulting text file is about 92 MB, up from 14 to start.

# Visualize

The rest of this notebook requires pyviz to be installed.

Let's see what we have!

In [ ]:
print(min(data),max(data))

In [ ]:
print(min(data_cal),max(data_cal))

Minimum and maximum values don't agree well. That's because the mag calibration also converts from Volts to microTesla, which ratio is as much as 100:1

Let's plot the time series before and after calibration:

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade, shade, dynspread
from bokeh.models.formatters import DatetimeTickFormatter
import panel as pn
import viz
hv.notebook_extension('bokeh','matplotlib')
pn.extension()

def verbose_formatter():
    vf = DatetimeTickFormatter()
    vf.microseconds = ['%f us']
    vf.milliseconds = ['%S.%3N s']
    # vf.milliseconds = ['%H:%M:%S.%3N']
    vf.seconds = ['%H:%M:%S']
    vf.minsec = ['%H:%M:%S']
    vf.minutes = ['%m/%d %H:%M']
    vf.hourmin = ['%m/%d %H:%M']
    vf.hours = ['%m/%d %H:%M']
    vf.days = ['%m/%d', '%a%d']
    vf.months = ['%m/%Y', '%b %Y']
    vf.years = ['%Y']
    return vf


ts_tools = ['save','pan','xwheel_zoom','box_zoom','undo','reset']
time = hv.Dimension('time',label='Time')
signal = hv.Dimension('signal',label='Signal',unit='V')
craw = hv.Curve((times,data),time,signal)
draw = dynspread(datashade(craw).opts(#framewise=True,
                                       xformatter=verbose_formatter(),
                                       default_tools=ts_tools)
                                      )
ccal = hv.Curve((times,data_cal*100),time,signal)
dcal = dynspread(datashade(ccal).opts(#framewise=True,
                                       xformatter=verbose_formatter(),
                                       default_tools=ts_tools)
                                      )
q = hv.Layout(draw+dcal).cols(1)
pn.Column(q)

Hmm. There's ringing at the start and end of the signal; to be expected. Also, the calibrated time signal looks much less smooth. Maybe that's just because the mag calibration is so dramatic? Let's plot that calibration.

In [ ]:
import hvplot.pandas

In [ ]:
byfreq = antcal.groupby(['antenna_sn','base_frequency']).sum()
byfreq[['mag']].hvplot('base_frequency',logx=True,logy=True,groupby='antenna_sn',height=500)

Frequencies below .03 and above 200 are being amplified by the calibration (calibrated signal = input / calibration, so frequencies are amplified where this plot has small values). That's what's changing the character of the time series - the low and high frequencies being amplified. Calibration curves are pretty similar across all coils. Let's look at the board calibration too, just to be safe.

In [ ]:
board_byfreq = cals.groupby(['CARDSNX','ADFREQ','CALFREQ']).sum()
board_byfreq[['MAG']].hvplot('CALFREQ',logx=True,by='ADFREQ',groupby=['CARDSNX'],height=500,xlim=(.006,10000))

Three curves, depending on sampling rate. Very little variance from card to card. Nyquist frequency for 4096 Hz sampling rate is 2048. Calibration magnitude at 2048 Hz is above 0.9, so this isn't what's causing the time series to look so strange.

How does the frequency content of the signal look before and after calibration?

In [ ]:
frequency = hv.Dimension('frequency',label='Frequency')
curve_ft = hv.Curve((np.log10(freqs[1:]),np.log10(np.abs(ft[1:]))),frequency,signal)
dyn_ft = dynspread(datashade(curve_ft).opts(
                                       width=500,
                                       default_tools=ts_tools)
                                      )
curve_ft_cal = hv.Curve((np.log10(freqs[1:]),np.log10(np.abs(ft_cal[1:]))),frequency,signal)
dyn_ft_cal = dynspread(datashade(curve_ft_cal).opts(
                                       width=500,
                                       default_tools=ts_tools)
                                      )
#curve_cal = byfreq.loc[2064].hvplot('base_frequency',logx=True,height=500)
curve_cal = hv.Curve((np.log10(byfreq.loc[2064].index.values),np.log10(byfreq.loc[2064].mag.values)))
q2 = hv.Layout((dyn_ft+dyn_ft_cal)*curve_cal).cols(1)
pn.Column(q2)

In [ ]:
byfreq.loc[2064].hvplot('base_frequency',logx=True,height=500)